In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
def no_accent_vietnamese(s):
    s = re.sub(u'[àáạảãâầấậẩẫăằắặẳẵ]', 'a', s)
    s = re.sub(u'[ÀÁẠẢÃĂẰẮẶẲẴÂẦẤẬẨẪ]', 'A', s)
    s = re.sub(u'[èéẹẻẽêềếệểễ]', 'e', s)
    s = re.sub(u'[ÈÉẸẺẼÊỀẾỆỂỄ]', 'E', s)
    s = re.sub(u'[òóọỏõôồốộổỗơờớợởỡ]', 'o', s)
    s = re.sub(u'[ÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠ]', 'O', s)
    s = re.sub(u'[ìíịỉĩ]', 'i', s)
    s = re.sub(u'[ÌÍỊỈĨ]', 'I', s)
    s = re.sub(u'[ùúụủũưừứựửữ]', 'u', s)
    s = re.sub(u'[ƯỪỨỰỬỮÙÚỤỦŨ]', 'U', s)
    s = re.sub(u'[ỳýỵỷỹ]', 'y', s)
    s = re.sub(u'[ỲÝỴỶỸ]', 'Y', s)
    s = re.sub(u'Đ', 'D', s)
    s = re.sub(u'đ', 'd', s)
    s = s.lower()
    return s

In [87]:
df = pd.read_excel('/content/Danh sách cấp tỉnh kèm theo quận huyện, phường xã ___15_03_2023.xls')
df = df.drop(['Tên Tiếng Anh'], axis=1)
# df[df['Tỉnh Thành Phố'] == "Thành phố Hải Phòng"]
df.info()

WARNING *** file size (2453055) not 512 + multiple of sector size (512)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10604 entries, 0 to 10603
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Tỉnh Thành Phố  10604 non-null  object 
 1   Mã TP           10604 non-null  int64  
 2   Quận Huyện      10604 non-null  object 
 3   Mã QH           10604 non-null  int64  
 4   Phường Xã       10599 non-null  object 
 5   Mã PX           10599 non-null  float64
 6   Cấp             10604 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 580.0+ KB


In [88]:
df.isna().any()

Tỉnh Thành Phố    False
Mã TP             False
Quận Huyện        False
Mã QH             False
Phường Xã          True
Mã PX              True
Cấp               False
dtype: bool

In [89]:
df[df.isna().any(axis=1)]

,Tỉnh Thành Phố,Mã TP,Quận Huyện,Mã QH,Phường Xã,Mã PX,Cấp
3797,Thành phố Hải Phòng,31,Huyện Bạch Long Vĩ,318,NaN,NaN,Huyện
6208,Tỉnh Quảng Trị,45,Huyện Cồn Cỏ,471,NaN,NaN,Huyện
6406,Thành phố Đà Nẵng,48,Huyện Hoàng Sa,498,NaN,NaN,Huyện
6821,Tỉnh Quảng Ngãi,51,Huyện Lý Sơn,536,NaN,NaN,Huyện
8686,Tỉnh Bà Rịa - Vũng Tàu,77,Huyện Côn Đảo,755,NaN,NaN,Huyện


In [72]:
df.columns

Index(['Tỉnh Thành Phố', 'Mã TP', 'Quận Huyện', 'Mã QH', 'Phường Xã', 'Mã PX',
       'Cấp', 'Tên Tiếng Anh'],
      dtype='object')

In [73]:
data = {"provincesCities": []}
provinces = df['Tỉnh Thành Phố'].unique()
for province in provinces:
    rslt_df = df[df['Tỉnh Thành Phố'] == province] 
    item = {
        "id": int(rslt_df.values[0][1]),
        "name": rslt_df.values[0][0],
        "slug": no_accent_vietnamese(rslt_df.values[0][0]).replace('-', '_').replace(' ', '_').replace('___', '_'),
        "district": []
    }
    districts = rslt_df['Quận Huyện'].unique()
    
    for district in districts:
        ward = df[df['Quận Huyện'] == district]
        
        item_district = {
            "id": int(ward.values[0][3]),
            "name": ward.values[0][2],
            "slug": no_accent_vietnamese(ward.values[0][2]).replace('-', '_').replace(' ', '_').replace('___', '_'),
            "ward": [],
        }
        for row in ward.values:
            try:
                item_ward = {
                "id": int(row[5]),
                "name": row[4],
                "slug": no_accent_vietnamese(row[4]).replace('-', '_').replace(' ', '_').replace('___', '_'),
                }
                item_district['ward'].append(item_ward)
            except:
                pass
        item['district'].append(item_district)
    data['provincesCities'].append(item)


In [75]:
import json
 
with open("data.json", "w") as outfile:
    json.dump(data, outfile, ensure_ascii=False)

In [90]:
{
    "id": 1,
    "name": "Thành phố Hà Nội",
    "slug": "Thành phố Hà Nội",
    "district": [],
}

{'id': 1,
 'name': 'Thành phố Hà Nội',
 'slug': 'Thành phố Hà Nội',
 'district': []}